In [ ]:
from __future__ import division, print_function, absolute_import

from skimage import color, io
from scipy.misc import imresize
import numpy as np
from sklearn.cross_validation import train_test_split
import os
from glob import glob

import tflearn
from tflearn.data_utils import shuffle, to_categorical
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation
from tflearn.metrics import Accuracy


In [ ]:
files_path = '/Users/shivanshmehta/train/'

cat_files_path = os.path.join(files_path, 'cat.*.jpg')
dog_files_path = os.path.join(files_path, 'dog.*.jpg')

cat_files = sorted(glob(cat_files_path))
dog_files = sorted(glob(dog_files_path))

n_files = len(cat_files) + len(dog_files)
print(n_files)

size_image = 64

allX = np.zeros((n_files, size_image, size_image, 3), dtype='float64')
ally = np.zeros(n_files)
count = 0
for f in cat_files:
    try:
        img = io.imread(f)
        new_img = imresize(img, (size_image, size_image, 3))
        allX[count] = np.array(new_img)
        ally[count] = 0
        count += 1
    except:
        continue

for f in dog_files:
    try:
        img = io.imread(f)
        new_img = imresize(img, (size_image, size_image, 3))
        allX[count] = np.array(new_img)
        ally[count] = 1
        count += 1
    except:
        continue

In [ ]:
X, X_test, Y, Y_test = train_test_split(allX, ally, test_size=0.1, random_state=42)


In [ ]:
Y = to_categorical(Y, 2)
Y_test = to_categorical(Y_test, 2)

In [ ]:
img_prep = ImagePreprocessing()
img_prep.add_featurewise_zero_center()
img_prep.add_featurewise_stdnorm()

# Create extra synthetic training data by flipping & rotating images
img_aug = ImageAugmentation()
img_aug.add_random_flip_leftright()
img_aug.add_random_rotation(max_angle=25.)

In [ ]:

network = input_data(shape=[None, 64, 64, 3],
                     data_preprocessing=img_prep,
                     data_augmentation=img_aug)


conv_1 = conv_2d(network, 32, 3, activation='relu', name='conv_1')


network = max_pool_2d(conv_1, 2)


conv_2 = conv_2d(network, 64, 3, activation='relu', name='conv_2')


conv_3 = conv_2d(conv_2, 64, 3, activation='relu', name='conv_3')


network = max_pool_2d(conv_3, 2)


network = fully_connected(network, 512, activation='relu')


network = dropout(network, 0.5)


network = fully_connected(network, 2, activation='softmax')


acc = Accuracy(name="Accuracy")
network = regression(network, optimizer='adam',
                     loss='categorical_crossentropy',
                     learning_rate=0.0005, metric=acc)

In [ ]:
model = tflearn.DNN(network, checkpoint_path='model_cat_dog_6.tflearn', max_checkpoints = 3,
                    tensorboard_verbose = 3, tensorboard_dir='tmp/tflearn_logs/')


In [ ]:
model.fit(X, Y, validation_set=(X_test, Y_test), batch_size=500,
      n_epoch=30, run_id='model_cat_dog_6', show_metric=True)